<h2>Cálculo da solução ótima</h2>
<br>
<br>
Através da curva ABC, iremos trabalhar com <strong>20% dos produtos</strong> da farmácia que representam cerca de <strong>80% da soma total anual</strong> dos remédios da farmácia.<br>
Dito isso, nossa amostra serão os produtos das linhas <strong>1 a 562.</strong> Com a quantidade de linhas de 2118. 


In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pulp

    100% |████████████████████████████████| 13.6MB 2.3MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


In [3]:
#Bibliotecas
import pandas as pd
from pulp import *

In [72]:
#Entrando com a tabela
dados = pd.read_csv("relatorio.csv").head(562)
dados.head(10)

,PRODUTO_ID,NCM,EAN,DESCRICAO,APRESENTACAO,QUANTIDADE,TOTAL_BRUTO,TOTAL_LIQUIDO,PROD_SALDO,VALOR_ULT_COMPRA,Unnamed: 10,SOMA TOTAL,Produtos A,Produtos B,Produtos C
0,21081,30049079.0,7.891320e+12,CITRATO DE SILDENAFILA 50MG COMP,50mg cx 8 comp rev,100,12426.600190,6213.210075,4.0,4.105300,NaN,"$530,724.79",80.00%,15.00%,5.00%
1,8501,30049037.0,7.896710e+12,TORSILAX C/ 10 COMPR. NEO QUIM.,CX/ 30 COMPR.,886,10289.360200,5178.870199,-12.0,2.290889,NaN,NaN,NaN,NaN,NaN
2,17371,30049069.0,7.896640e+12,NEOSALDINA 30 DRAG,NaN,315,9293.980021,7436.830130,13.0,22.237499,NaN,NaN,Valor Limite,NaN,NaN
3,8912,30039049.0,7.896520e+12,CIMELIDE C/12 COMPR.(NIMESULIDA),NIMESULIDA CIMED,497,7971.510023,4008.039969,71.0,2.524600,NaN,NaN,"$424,579.83","$79,608.72","$26,536.24"
4,19551,30049039.0,7.891060e+12,DORFLEX,300+35+50 mg comp x 300 emb. mult,51,7811.520081,4922.370056,1.0,117.339996,NaN,NaN,NaN,NaN,NaN
5,13028,30049079.0,7.891720e+12,MELOXICAM 15 MG.C/10 COMP. MERCK,GENERICO MERCK,134,4978.990086,2510.040016,5.0,6.455000,NaN,Linha inicial,Linha Limite,NaN,NaN
6,16367,30049067.0,7.896710e+12,CITRATO DE SILDENAFILA,50mg cx 4 comp,196,4653.140087,3124.250049,49.0,3.320000,NaN,1,562,1200,2119
7,18412,30049099.0,7.896470e+12,MULTIGRIP,400+4+4mg 2bl x 10 cap,198,4108.739998,2059.699992,33.0,4.290000,NaN,NaN,562,1200,2119
8,16276,30049099.0,7.898160e+12,FLORENT,100MG CX 12 CAP,104,3844.139931,1964.170044,6.0,12.188750,NaN,NaN,NaN,NaN,NaN
9,18969,30049077.0,7.899100e+12,BETRICORT,creme bg 30g,145,3826.599955,1915.829967,17.0,5.310000,NaN,Quantidade Linhas,2118,NaN,NaN


In [73]:
dados_parciais = []
dados_id = dados.PRODUTO_ID
dados_parciais.insert(0,pd.read_csv("agosto2018.csv", sep=';'))
dados_parciais.insert(1,pd.read_csv("outubro2018.csv", sep=';'))
dados_parciais.insert(2,pd.read_csv("dezembro2018.csv", sep=';'))
dados_parciais.insert(3,pd.read_csv("fevereiro2019.csv", sep=';'))
dados_parciais.insert(4,pd.read_csv("abril2019.csv", sep=';'))
dados_parciais.insert(5,pd.read_csv("junho2019.csv", sep=';'))

In [69]:
dados_parciais[0].tail(10)

,Cod. Barras,NCM,Codigo,Produto,Apresentacao,Quantidade,Total Bruto,Desconto,Total Liquido,Ulti. Compra,Saldo Atual
686,7896641804465,30049099,20618,XANTINON 100COMP,drg ct 10 bl x 10 emb mult,1,"R$ 27,74","R$ 5,55","R$ 22,19","R$ 22,11",1
687,7590002023228,30049029,10346,XAROPE VICK 44E 120 ML,VICK 44E 120 ML,2,"R$ 43,22","R$ 8,64","R$ 34,58","R$ 14,54",5
688,7896112480709,30049079,17715,XEFO,8mg cx 20 comp rev,1,"R$ 23,08","R$ 0,00","R$ 23,08","R$ 17,34",2
689,5000456028387,30049049,20990,XIGDUO XR 10+1000MG 30COMP,10+1000mg cx 30 comp rev,1,"R$ 182,80","R$ 45,70","R$ 137,10","R$ 141,18",0
690,7897947613645,38089199,20559,XO INSETO 15% LOC FR 200 ML,NaN,2,"R$ 20,00","R$ 0,00","R$ 20,00","R$ 0,60",15
691,7897572006171,30049059,20965,ZETSIM 10/40MG 30 COMP,10+40mg cx 30 comp,1,"R$ 130,11","R$ 26,02","R$ 104,09","R$ 93,34",0
692,7891317457211,30049069,15964,ZINA,5MG,1,"R$ 37,95","R$ 7,59","R$ 30,36","R$ 29,70",2
693,7895858010331,30049069,20190,ZYLORIC,300mg cx 30 comp,1,"R$ 46,05","R$ 9,21","R$ 36,84","R$ 33,42",1
694,7897316805442,30049099,17093,ZYPRED,fr 6ml,1,"R$ 65,00","R$ 13,00","R$ 52,00","R$ 46,02",0
695,5413787037797,30049069,18565,ZYXEM,5mg fr gts 20ml,2,"R$ 112,58","R$ 22,52","R$ 90,06","R$ 45,35",2


In [97]:
quantidades = [[0 for x in range(6)] for y in range(len(dados_id))] 
for index,item in enumerate(dados_id):
    for i in range(6):
        print(item in dados_parciais[i].Codigo)
        if (item in dados_parciais[i].Codigo):
            ind = dados_parciais[i].Codigo.index[item]
            quantidades[index].insert(index,float(dados_parciais[i].Quantidade[ind]))
        else:
            break      
print(quantidades[45])
print(dados.DESCRICAO[45])


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [5]:
limite = 562

In [303]:
#Esse vetor possui todos os valores (TOTAL_BRUTO) dos produtos pertencentes a classe A
#Os valores desse vetor posteriormente vão ser trabalhados
valorClasseA = []
for index,item in enumerate(dados.TOTAL_BRUTO):
    if(',' in item):
        lista_modifica_string = list(item)
        index_virgula = lista_modifica_string.index(',')
        lista_modifica_string[index_virgula] = '.'
        new_string = "".join(lista_modifica_string)
    else:
        new_string = item
    if (index < limite):
        valorClasseA.insert(index,float(new_string))
    else:
        break

In [304]:
#Esse vetor possui todos os valores (QUANTIDADE) dos produtos pertencentes a classe A
#Os valores desse vetor posteriormente vão ser trabalhados
quantidadeClasseA = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        quantidadeClasseA.insert(index,item)
    else:
        break

In [305]:
quantidadeClasseA

[100,
 886,
 315,
 497,
 51,
 134,
 196,
 198,
 104,
 145,
 267,
 426,
 119,
 4,
 187,
 306,
 29,
 27,
 260,
 156,
 45,
 144,
 15,
 259,
 46,
 37,
 114,
 64,
 49,
 122,
 15,
 26,
 73,
 36,
 111,
 52,
 49,
 8,
 85,
 9,
 18,
 27,
 33,
 14,
 65,
 295,
 27,
 45,
 104,
 86,
 18,
 205,
 19,
 69,
 90,
 46,
 5,
 39,
 48,
 17,
 131,
 64,
 44,
 126,
 177,
 38,
 111,
 19,
 25,
 25,
 10,
 52,
 22,
 49,
 11,
 4,
 72,
 45,
 15,
 36,
 52,
 1147,
 52,
 13,
 9,
 456,
 93,
 5,
 16,
 34,
 40,
 4,
 58,
 13,
 77,
 18,
 13,
 24,
 225,
 62,
 38,
 31,
 36,
 302,
 35,
 14,
 49,
 29,
 17,
 7,
 14,
 45,
 43,
 28,
 14,
 6,
 71,
 23,
 17,
 73,
 19,
 7,
 10,
 7,
 21,
 36,
 6,
 7,
 21,
 23,
 224,
 16,
 416,
 24,
 15,
 48,
 41,
 9,
 39,
 7,
 8,
 9,
 107,
 24,
 45,
 8,
 56,
 17,
 12,
 12,
 19,
 6,
 38,
 63,
 41,
 33,
 9,
 33,
 6,
 26,
 92,
 39,
 46,
 45,
 33,
 25,
 49,
 75,
 5,
 3,
 315,
 14,
 34,
 38,
 132,
 7,
 39,
 57,
 6,
 10,
 18,
 10,
 19,
 65,
 29,
 9,
 25,
 41,
 53,
 39,
 29,
 6,
 8,
 7,
 27,
 7,
 4,
 17,
 24,

In [306]:
mediaVenda = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        media = quantidadeClasseA[index] / 12
        mediaVenda.insert(index,media)
    else:
        break

In [128]:
mediaVenda

[8.333333333333334,
 73.83333333333333,
 26.25,
 41.416666666666664,
 4.25,
 11.166666666666666,
 16.333333333333332,
 16.5,
 8.666666666666666,
 12.083333333333334,
 22.25,
 35.5,
 9.916666666666666,
 0.3333333333333333,
 15.583333333333334,
 25.5,
 2.4166666666666665,
 2.25,
 21.666666666666668,
 13.0]

In [307]:
lucro = []
for index,item in enumerate(dados.TOTAL_LIQUIDO):
    if(',' in item):
        lista_modifica_string = list(item)
        index_virgula = lista_modifica_string.index(',')
        lista_modifica_string[index_virgula] = '.'
        new_string = "".join(lista_modifica_string)
    else:
        new_string = item
    if (index < limite):
        lucro.insert(index, float(new_string)/quantidadeClasseA[index])
    else:
        break

In [308]:
lucro[0:20]

[62.13210075,
 5.845225958239277,
 23.60898453968254,
 8.064466738430584,
 96.51705992156862,
 18.73164191044776,
 15.940051270408164,
 10.402525212121212,
 18.886250423076923,
 13.212620462068966,
 11.659737692883896,
 4.336995340375587,
 15.580756537815127,
 426.600006,
 8.053903647058823,
 5.733398663398693,
 37.636206517241376,
 51.94555622222222,
 5.015269084615384,
 13.363333282051281]

In [309]:
desvioPadrao = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        desvioPadrao.insert(index, mediaVenda[index] * 0.1)
    else:
        break

In [310]:
vendaMinima = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        vendaMinima.insert(index, (mediaVenda[index] - desvioPadrao[index]))
    else:
        break

In [370]:
vendaMaxima = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        vendaMaxima.insert(index, (mediaVenda[index] + desvioPadrao[index]))
    else:
        break

In [282]:
giroEstoque = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        giroEstoque.insert(index, float(item)/(float(item) + float(dados.PROD_SALDO[index])))
    else:
        break

In [312]:
custoUnitario = []
for index,item in enumerate(dados.TOTAL_LIQUIDO):
    if(',' in item):
        lista_modifica_string = list(item)
        index_virgula = lista_modifica_string.index(',')
        lista_modifica_string[index_virgula] = '.'
        new_string = "".join(lista_modifica_string)
    else:
        new_string = item
    if (index < limite):
        custoUnitario.insert(index, (valorClasseA[index]-float(new_string))/quantidadeClasseA[index])
    else:
        break

In [232]:
custoUnitario[0:20]

[62.13390114999999,
 5.768047405191873,
 5.895713939682537,
 7.974788841046277,
 56.65000049019608,
 18.425000522388057,
 7.800459377551018,
 10.3486868989899,
 18.076633528846155,
 13.177724055172416,
 2.63505629588015,
 4.21725372769953,
 14.819664092436975,
 426.592499,
 7.758823427807488,
 3.8066012973856203,
 61.13379013793103,
 51.94555622222222,
 5.015384473076923,
 3.256859089743592]

In [435]:
capital = 0
for index,item in enumerate(dados.TOTAL_LIQUIDO):
    if(',' in item):
        lista_modifica_string = list(item)
        index_virgula = lista_modifica_string.index(',')
        lista_modifica_string[index_virgula] = '.'
        new_string = "".join(lista_modifica_string)
    else:
        new_string = item
    if (index < limite):
        capital += valorClasseA[index]-float(new_string)
    else:
        break

In [436]:
capital

141215.60983940001

In [499]:
#Cria o problema
prob = LpProblem("Maximização de lucros - Farmácia", LpMaximize)

In [500]:
#Cria as variáveis
xi = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        xi.insert(index, LpVariable("q"+str(index),0,None))
    else:
        break

In [501]:
obj = None
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        obj += lucro[index]*xi[index]
    else:
        break

In [502]:
#Função objetivo
prob += obj

In [503]:
#Restrições - Venda Minima
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        prob += xi[index] >= vendaMinima[index]
    else:
        break

In [504]:
#Restrições - Venda Maxima
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        prob += xi[index] <= float(item) + (12*desvioPadrao[index])
    else:
        break

In [505]:
#Restrições - Espaço
obj = None
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        obj += xi[index]
    else:
        break

In [506]:
prob+= obj <= 21031

In [508]:
prob.solve()

1

In [509]:
#Imprimindo o status da resolucao
print("Status: ", LpStatus[prob.status])

Status:  Optimal


In [510]:
#Imprimindo os valores das variáveis
for cont in prob.variables():
    print(cont.name, " = ", cont.varValue)

q0  =  110.0
q1  =  974.6
q10  =  293.7
q100  =  41.8
q101  =  34.1
q102  =  39.6
q103  =  332.2
q104  =  38.5
q105  =  15.4
q106  =  53.9
q107  =  31.9
q108  =  18.7
q109  =  7.7
q11  =  468.6
q110  =  15.4
q111  =  49.5
q112  =  47.3
q113  =  30.8
q114  =  15.4
q115  =  6.6
q116  =  78.1
q117  =  25.3
q118  =  18.7
q119  =  80.3
q12  =  130.9
q120  =  20.9
q121  =  7.7
q122  =  11.0
q123  =  7.7
q124  =  23.1
q125  =  39.6
q126  =  6.6
q127  =  7.7
q128  =  23.1
q129  =  25.3
q13  =  4.4
q130  =  246.4
q131  =  17.6
q132  =  268.175
q133  =  26.4
q134  =  16.5
q135  =  52.8
q136  =  45.1
q137  =  9.9
q138  =  42.9
q139  =  7.7
q14  =  205.7
q140  =  8.8
q141  =  9.9
q142  =  117.7
q143  =  26.4
q144  =  49.5
q145  =  8.8
q146  =  61.6
q147  =  18.7
q148  =  13.2
q149  =  13.2
q15  =  336.6
q150  =  20.9
q151  =  6.6
q152  =  41.8
q153  =  69.3
q154  =  45.1
q155  =  36.3
q156  =  9.9
q157  =  36.3
q158  =  6.6
q159  =  28.6
q16  =  31.9
q160  =  101.2
q161  =  42.9
q162  =  50.6
q163

In [512]:
custo = 0
#Calculo Custo Total
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        custo += custoUnitario[index]*prob.variables()[index].varValue
    else:
        break
print(custo)

465320.7912558254


In [514]:
custo = 0
#Calculo Custo Total
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        custo += prob.variables()[index].varValue
    else:
        break
print(custo)

21031.00000000001


In [513]:
#Imprimindo o valor do custo total
print("Lucro total: ", value(prob.objective))

Lucro total:  310013.84255884937
